In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/exo-data/testing.npy
/kaggle/input/exo-data/error_bars.npy
/kaggle/input/exo-data/training.npy
/kaggle/input/exo-data/wavelength.npy
/kaggle/input/exo-data/WASP12b.npy


In [2]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 8.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.4/256.4 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 59.2 MB/s eta 0:00:0000:0100:01
  Using cached importlib_metadata-6.4.1-py3-none-any.whl (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.9 MB/s eta 0:00:00
     

In [3]:
from pycaret.regression import *
import pandas as pd

In [4]:
data = np.load("/kaggle/input/exo-data/training.npy")
data2 = np.load("/kaggle/input/exo-data/testing.npy")
pred_data = np.load("/kaggle/input/exo-data/WASP12b.npy")


In [5]:
data1 = data[:,:14]
df1 = pd.DataFrame(data1)
data3 = data2[:,:14]
testdata = pd.DataFrame(data3)
new_data = pd.DataFrame(pred_data)

In [ ]:
s = setup(data = df1, train_size = 0.8 , test_data = testdata, target='13', session_id = 123, use_gpu=True, pca=True,pca_method='incremental',
          normalize = True, normalize_method = 'minmax')

In [ ]:
xgboost = create_model('xgboost', fold=10) #kernel

In [8]:
xgboost = create_model('xgboost', fold=10) #incremental

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,224.9079,115985.0469,340.5658,0.7587,0.2442,0.1697
1,220.7719,111741.7969,334.2780,0.7670,0.2404,0.1663
2,223.0598,117124.3359,342.2343,0.7597,0.2471,0.1703
3,225.6494,118278.7578,343.9168,0.7534,0.2460,0.1695
4,224.8697,116714.5703,341.6351,0.7552,0.2446,0.1691
5,224.2187,116540.4141,341.3802,0.7582,0.2438,0.1687
6,223.2168,113564.3516,336.9931,0.7646,0.2450,0.1712
7,223.5858,114425.2422,338.2680,0.7598,0.2461,0.1712
8,228.4742,122157.4766,349.5103,0.7461,0.2477,0.1707


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
compare_tree_models = compare_models(include = [ 'xgboost', 'lightgbm', 'catboost'],fold=5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,225.0713,114256.3701,337.9972,0.7621,0.2419,0.1692,20.7360
lightgbm,Light Gradient Boosting Machine,224.8420,114468.1813,338.3106,0.7617,0.2435,0.1706,2.1380
xgboost,Extreme Gradient Boosting,225.3690,117644.6172,342.9655,0.7550,0.2459,0.1700,0.7740


Processing:   0%|          | 0/17 [00:00<?, ?it/s]

In [10]:
tuned_model_xg = load_model('/kaggle/input/boosting-models-tuning/tuned_xgboost_model')
print(tuned_model_xg)

Transformation Pipeline and Model Successfully Loaded
Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['0', '1', '2', '3', '4', '5', '6',
                                             '7', '8', '9', '10', '11', '12'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('normalize', TransformerWrapper(transformer=MinMaxSc...
                              gamma=0, gpu_id=0, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.05, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=11, max_leaves=0, min_child_weight=3,
                

In [12]:
tuned_xgbr, tuner = tune_model(xgboost, return_tuner=True, fold=10,n_iter = 10 , return_train_score=True)

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [11]:
tunedxgboost = create_model(tuned_model_xg, fold=10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,229.9426,119923.3672,346.2995,0.7505,0.2464,0.1728
1,226.3598,115300.9688,339.5600,0.7596,0.2438,0.1706
2,228.7386,117541.4141,342.8431,0.7588,0.2457,0.1737
3,232.7161,122571.1875,350.1017,0.7445,0.2485,0.1734
4,232.0750,121868.1094,349.0961,0.7444,0.2490,0.1740
5,231.0892,119418.9062,345.5704,0.7523,0.2465,0.1730
6,229.9702,117417.9609,342.6631,0.7566,0.2462,0.1748
7,230.0045,118630.5625,344.4279,0.7510,0.2477,0.1745
8,233.4470,126021.8125,354.9955,0.7381,0.2506,0.1741


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
predictions_tunedxg = predict_model(tuned_xgboost, data=new_data)

In [ ]:
predictions_tunedxg

### Water 

In [22]:
dataw1 = data[:,:14]
dataw1[:,13:14] = data[:,14:15]
dfw1 = pd.DataFrame(dataw1)
dataw2 = data2[:,:14]
dataw2[:,13:14] = data2[:,14:15]
testdataw = pd.DataFrame(dataw2)

In [25]:
sw = setup(data = dfw1, train_size = 1.0 , test_data = testdataw, target='13', session_id = 123, use_gpu=True, 
          normalize = True, normalize_method = 'minmax')

,Description,Value
0,Session id,123
1,Target,13
2,Target type,Regression
3,Original data shape,"(100000, 14)"
4,Transformed data shape,"(140002, 14)"
5,Transformed train set shape,"(100000, 14)"
6,Transformed test set shape,"(40002, 14)"
7,Numeric features,13
8,Preprocess,True
9,Imputation type,simple


In [26]:
xgboost = create_model('xgboost', fold=20)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.7624,5.7206,2.3918,0.5908,0.3722,0.7898
1,1.7845,5.8024,2.4088,0.5829,0.3834,2.5083
2,1.7928,5.8306,2.4147,0.5823,0.3782,0.8600
3,1.7760,5.8124,2.4109,0.5957,0.3844,6.3608
4,1.7744,5.7295,2.3936,0.5988,0.3713,2.3657
5,1.7794,5.7633,2.4007,0.5906,0.3791,2.1822
6,1.7398,5.6280,2.3723,0.5967,0.3721,1.1158
7,1.8234,6.0064,2.4508,0.5824,0.3896,1.1172
8,1.7873,5.7520,2.3983,0.5918,0.3707,0.8852


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [27]:
tunedxgboost = create_model(tuned_model_xg, fold=20)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.6878,5.4579,2.3362,0.6096,0.3542,0.6902
1,1.7115,5.5452,2.3548,0.6014,0.3674,2.1771
2,1.7170,5.5937,2.3651,0.5992,0.3613,0.8576
3,1.6994,5.5468,2.3552,0.6142,0.3671,5.9988
4,1.6899,5.4827,2.3415,0.6161,0.3545,2.0230
5,1.7027,5.4900,2.3431,0.6100,0.3612,1.7898
6,1.6686,5.3880,2.3212,0.6139,0.3581,1.0679
7,1.7364,5.7201,2.3917,0.6023,0.3731,1.0468
8,1.7103,5.5379,2.3533,0.6070,0.3563,0.8496


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [30]:
# ensemble with bagging
ensemble_model(xgboost, method = 'Bagging')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.7472,5.5801,2.3622,0.6000,0.3715,1.5321
1,1.7577,5.6518,2.3773,0.6011,0.3747,3.8393
2,1.7475,5.5628,2.3586,0.6077,0.3688,2.2771
3,1.7538,5.6452,2.3760,0.6016,0.3758,1.1332
4,1.7641,5.6041,2.3673,0.6031,0.3676,0.9441
5,1.7512,5.5735,2.3608,0.6015,0.3721,1.4386
6,1.7711,5.6804,2.3834,0.5922,0.3742,1.2419
7,1.7688,5.6882,2.3850,0.5952,0.3777,1.2681
8,1.7679,5.6125,2.3691,0.5972,0.3738,4.3262


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

BaggingRegressor(base_estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                             callbacks=None,
                                             colsample_bylevel=1,
                                             colsample_bynode=1,
                                             colsample_bytree=1,
                                             early_stopping_rounds=None,
                                             enable_categorical=False,
                                             eval_metric=None, gamma=0,
                                             gpu_id=0, grow_policy='depthwise',
                                             importance_type=None,
                                             interaction_constraints='',
                                             learning_rate=0.300000012,
                                             max_bin=256, max_cat_to_onehot=4,
                                             max_delta_step=0, max_depth=6,
 

### HCN

In [42]:
datac1 = data[:,:14]
datac1[:,13:14] = data[:,15:16]
dfc1 = pd.DataFrame(datac1)
datac2 = data2[:,:14]
datac2[:,13:14] = data2[:,15:16]
testdatac = pd.DataFrame(datac2)

In [43]:
scn = setup(data = dfc1, train_size = 1.0 , test_data = testdatac, target='13', session_id = 123, use_gpu=True, 
          normalize = True, normalize_method = 'minmax')

,Description,Value
0,Session id,123
1,Target,13
2,Target type,Regression
3,Original data shape,"(100000, 14)"
4,Transformed data shape,"(140002, 14)"
5,Transformed train set shape,"(100000, 14)"
6,Transformed test set shape,"(40002, 14)"
7,Numeric features,13
8,Preprocess,True
9,Imputation type,simple


In [44]:
xgboost3 = create_model('xgboost', fold=20)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.1083,7.5979,2.7564,0.4606,0.4574,2.2523
1,2.1510,7.7967,2.7923,0.4443,0.4650,2.8469
2,2.1377,7.7327,2.7808,0.4535,0.4544,1.5103
3,2.1311,7.6253,2.7614,0.4436,0.4467,2.6523
4,2.1381,7.6890,2.7729,0.4549,0.4609,2.6428
5,2.0934,7.5730,2.7519,0.4633,0.4500,1.2262
6,2.1336,7.7470,2.7833,0.4392,0.4479,4.0578
7,2.1010,7.4606,2.7314,0.4586,0.4412,1.0129
8,2.1293,7.7672,2.7870,0.4454,0.4545,1.2066


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [45]:
tunedxgboost3 = create_model(tuned_model_xg, fold=20)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.0559,7.4656,2.7323,0.4700,0.4429,1.8192
1,2.0956,7.6454,2.7650,0.4551,0.4483,3.1544
2,2.0730,7.5038,2.7393,0.4697,0.4375,1.4160
3,2.0382,7.2531,2.6932,0.4708,0.4255,1.9118
4,2.0674,7.4523,2.7299,0.4717,0.4448,1.8740
5,2.0260,7.3147,2.7046,0.4816,0.4305,1.0906
6,2.0618,7.4731,2.7337,0.4590,0.4298,3.3441
7,2.0350,7.2211,2.6872,0.4760,0.4263,0.9385
8,2.0541,7.4585,2.7310,0.4674,0.4356,1.1041


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

### NH3

In [46]:
datac1 = data[:,:14]
datac1[:,13:14] = data[:,16:17]
dfc1 = pd.DataFrame(datac1)
datac2 = data2[:,:14]
datac2[:,13:14] = data2[:,16:17]
testdatac = pd.DataFrame(datac2)

In [47]:
snh = setup(data = dfc1, train_size = 1.0 , test_data = testdatac, target='13', session_id = 123, use_gpu=True, 
          normalize = True, normalize_method = 'minmax')

,Description,Value
0,Session id,123
1,Target,13
2,Target type,Regression
3,Original data shape,"(100000, 14)"
4,Transformed data shape,"(140002, 14)"
5,Transformed train set shape,"(100000, 14)"
6,Transformed test set shape,"(40002, 14)"
7,Numeric features,13
8,Preprocess,True
9,Imputation type,simple


In [48]:
xgboost3 = create_model('xgboost', fold=20)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.3984,5.2267,2.2862,0.6811,0.3277,0.6875
1,1.4348,5.3655,2.3164,0.6726,0.3379,1.4994
2,1.3770,4.8809,2.2093,0.6971,0.3170,0.5472
3,1.3993,5.1376,2.2666,0.6823,0.3274,3.7441
4,1.4199,5.2014,2.2807,0.6856,0.3259,0.5754
5,1.4207,5.3433,2.3116,0.6696,0.3332,0.9856
6,1.3876,5.1517,2.2697,0.6795,0.3199,0.6566
7,1.4752,5.4801,2.3410,0.6606,0.3290,0.7297
8,1.4094,5.1805,2.2761,0.6780,0.3283,1.0665


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [49]:
tunedxgboost3 = create_model(tuned_model_xg, fold=20)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.2980,4.8965,2.2128,0.7012,0.3107,0.6307
1,1.3242,4.8927,2.2119,0.7015,0.3152,1.3214
2,1.2638,4.4145,2.1011,0.7260,0.2926,0.4824
3,1.2896,4.7370,2.1765,0.7071,0.3092,2.0034
4,1.3138,4.8704,2.2069,0.7056,0.3052,0.4963
5,1.3095,4.9004,2.2137,0.6970,0.3114,0.8830
6,1.2960,4.8475,2.2017,0.6985,0.3027,0.6552
7,1.3746,5.0935,2.2569,0.6845,0.3100,0.5981
8,1.3010,4.8008,2.1911,0.7016,0.3051,0.7976


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

### Opacity 

In [50]:
datac1 = data[:,:14]
datac1[:,13:14] = data[:,17:18]
dfc1 = pd.DataFrame(datac1)
datac2 = data2[:,:14]
datac2[:,13:14] = data2[:,17:18]
testdatac = pd.DataFrame(datac2)

In [51]:
snh = setup(data = dfc1, train_size = 1.0 , test_data = testdatac, target='13', session_id = 123, use_gpu=True, 
          normalize = True, normalize_method = 'minmax')

,Description,Value
0,Session id,123
1,Target,13
2,Target type,Regression
3,Original data shape,"(100000, 14)"
4,Transformed data shape,"(140002, 14)"
5,Transformed train set shape,"(100000, 14)"
6,Transformed test set shape,"(40002, 14)"
7,Numeric features,13
8,Preprocess,True
9,Imputation type,simple


In [52]:
xgboost3 = create_model('xgboost', fold=20)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.7399,5.0103,2.2384,0.7350,0.3893,2.4217
1,1.7359,5.0240,2.2414,0.7329,0.3983,1.9339
2,1.7377,4.9944,2.2348,0.7318,0.3859,0.9035
3,1.7437,5.0668,2.2510,0.7281,0.3947,1.1301
4,1.7470,5.0264,2.2420,0.7255,0.3854,1.0002
5,1.7657,5.1209,2.2629,0.7326,0.3915,1.4006
6,1.7304,4.9304,2.2204,0.7395,0.3953,1.1296
7,1.7348,5.0583,2.2491,0.7284,0.3908,0.9096
8,1.7479,5.0754,2.2529,0.7354,0.3919,3.7012


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [53]:
tunedxgboost3 = create_model(tuned_model_xg, fold=20)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.6891,4.8491,2.2021,0.7435,0.3793,2.3466
1,1.6876,4.9075,2.2153,0.7391,0.3901,1.5234
2,1.6990,4.8303,2.1978,0.7406,0.3761,0.8732
3,1.7103,4.9617,2.2275,0.7338,0.3832,1.0823
4,1.7043,4.8759,2.2082,0.7337,0.3775,0.9595
5,1.7165,4.9873,2.2332,0.7396,0.3803,1.0759
6,1.6891,4.8217,2.1958,0.7453,0.3847,1.0756
7,1.6954,4.9401,2.2226,0.7348,0.3805,0.9086
8,1.7064,4.9507,2.2250,0.7419,0.3818,3.5729


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [57]:
tuned_xgboostopa, tuner = tune_model(xgboost3,fold=10,return_tuner=True, n_iter = 10, return_train_score=True)

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
